In [162]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pdfplumber
import pandas as pd
import os

page_url = "http://complaints.cuapb.org/police_archive/browse/A/"

resp = requests.get(page_url)
resp.raise_for_status()
soup = BeautifulSoup(resp.text, "html.parser")


In [229]:
from requests.exceptions import HTTPError, RequestException
w=the_big_loop("http://complaints.cuapb.org/police_archive/")

http://complaints.cuapb.org/police_archive/browse/A
http://complaints.cuapb.org/police_archive/browse/B
http://complaints.cuapb.org/police_archive/browse/C


KeyboardInterrupt: 

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import fitz
from requests.exceptions import HTTPError
from concurrent.futures import ThreadPoolExecutor, as_completed

def alphabet_page_compiler(main_url):
    response=requests.get(main_url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "html.parser")
    a=soup.find_all("a")
    url_list=[]
    for k in a:
        url=k.get("href")
        pattern=re.compile(r"/police_archive/browse/")
        if pattern.search(url):
            url_list.append("http://complaints.cuapb.org"+url)
    return url_list


def pdf_scraper(url):
    if url=="No articles found":
        return "No articles found"

    try:
        response = requests.get(url)
        response.raise_for_status()
    except HTTPError:
        return("Access denied")
    pdf = response.content
    doc = fitz.open("pdf", pdf)
    text_parts = []
    if len(doc)>20:
        text_parts.append("Too_Long")
    for k in range(min(len(doc), 20)):
      text_parts.append(doc[k].get_text())
    text = "".join(text_parts)
    if len(text)==0:
        return "No text found"
    return text

def sub_url_officer_scraper(sub_url):
    response=requests.get(sub_url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "html.parser")
    a=soup.find_all("a")
    officer_url=[]
    for k in a:
        url=k.get("href")
        pattern=re.compile(r"/police_archive/officer/")
        if pattern.search(url):
            officer_url.append("http://complaints.cuapb.org"+url)
    return officer_url

def article_url_getter(officer_url):
    response=requests.get(officer_url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "html.parser")
    a=soup.find_all("a")
    article_list=[]
    for k in a:
        url=k.get("href")
        pattern=re.compile(r".pdf")
        if pattern.search(url):
            article_list.append(url)
    if len(article_list)==0:
        article_list.append("No articles found")
    return article_list
def officer_name(officer_url):
    url=officer_url
    resp=requests.get(url)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")
    for k in soup.find_all("h2"):
        #if "," in k:
        text = k.get_text(strip=True)
        pattern = re.compile(r", badge #")
        if pattern.search(text):
            return(text)
def department(officer_url):
    url=officer_url
    resp=requests.get(url)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")
    for k in soup.find_all("p"):
        #if "," in k:
        text = k.get_text(strip=True)
        #if pattern.search(text):
        return(text)
def the_big_loop(main_url):
    #df=pd.DataFrame(columns=["name", "department","url", "text"])
    records=[]
    alphabet_urls=alphabet_page_compiler(main_url)
    pairs=[]
    for letter in alphabet_urls:
        officer_url=[]
        print(letter)
        officer_for_letter=sub_url_officer_scraper(letter)
        for officer_url in officer_for_letter:
            #dm=pd.DataFrame(columns=["name", "department","url", "text"])
            name=officer_name(officer_url)
            off_department=department(officer_url)
            article_list=article_url_getter(officer_url)
            for article in article_list:
                pairs.append((name,off_department,article))
    future_to_meta = {}
    with ThreadPoolExecutor(max_workers=40) as executor:
        for name,off_department,url in pairs:
            future = executor.submit(pdf_scraper, url)
            future_to_meta[future] = {"name": name, "department": off_department, "url": url}
        for future in as_completed(future_to_meta):
            meta = future_to_meta[future]
            meta["text"] = future.result()
            records.append(meta)

    #df = pd.DataFrame(records)
    return pd.DataFrame(records)

In [31]:
a=the_big_loop("http://complaints.cuapb.org/police_archive/")

http://complaints.cuapb.org/police_archive/browse/A
http://complaints.cuapb.org/police_archive/browse/B
http://complaints.cuapb.org/police_archive/browse/C
http://complaints.cuapb.org/police_archive/browse/D
http://complaints.cuapb.org/police_archive/browse/E
http://complaints.cuapb.org/police_archive/browse/F
http://complaints.cuapb.org/police_archive/browse/G
http://complaints.cuapb.org/police_archive/browse/H
http://complaints.cuapb.org/police_archive/browse/I
http://complaints.cuapb.org/police_archive/browse/J
http://complaints.cuapb.org/police_archive/browse/K
http://complaints.cuapb.org/police_archive/browse/L
http://complaints.cuapb.org/police_archive/browse/M
http://complaints.cuapb.org/police_archive/browse/N
http://complaints.cuapb.org/police_archive/browse/O
http://complaints.cuapb.org/police_archive/browse/P
http://complaints.cuapb.org/police_archive/browse/Q
http://complaints.cuapb.org/police_archive/browse/R
http://complaints.cuapb.org/police_archive/browse/S
http://compl

In [ ]:
def clean_text(text):
    return text.replace("\n", " ").replace("\r", " ").replace("\t", " ").strip()
def cleaner_text(text):
    return re.sub(r"\s+", " ", text).strip()
a["text"] = a["text"].apply(cleaner_text)


NameError: name 'a' is not defined

In [ ]:
from ftfy import fix_text

a["text"] = a["text"].apply(lambda x: fix_text(x) if isinstance(x, str) else x)


In [34]:
a.to_csv("scraped_police_reports14.csv", index=False)